In [32]:
from trino.dbapi import connect
import pandas as pd

conn = connect(host="trino.trino", port=8080, user="user")

cur = conn.cursor()
cur.execute("select * from jtopen.demo.fraud limit 3")
rows = cur.fetchall()
rows
columns = [i.name for i in cur.description]
print(columns)
df = pd.DataFrame(rows, columns=[i.name for i in cur.description])
df.head()

['user', 'card', 'year', 'month', 'day', 'time', 'amount', 'use chip', 'merchant name', 'merchant city', 'merchant state', 'zip', 'mcc', 'errors?', 'is fraud?']


,user,card,year,month,day,time,amount,use chip,merchant name,merchant city,merchant state,zip,mcc,errors?,is fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,...,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No


In [6]:
# !pip install sklearn_pandas

In [1]:
import math
import os
import numpy as np
import pandas as pd
import re
from requests import get
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    FunctionTransformer,
    MinMaxScaler,
    LabelBinarizer,
)
from sklearn_pandas import DataFrameMapper
import csv

%load_ext lab_black

## 1) Preprocess Dataset using raw csv

In [22]:
f = get("https://ibm.box.com/shared/static/wamc5d0yve71jm46ntdp23q6xc001ew2.csv")
with open("card_transaction.v1.csv", "wb") as fd:
    fd.write(f.content)

dist = pd.DataFrame({"No": [], "Yes": []})
df_nf = pd.DataFrame()
df_f = pd.DataFrame()

with pd.read_csv(
    f"{os.getenv('HOME')}/card_transaction.v1.csv", chunksize=1_000_000
) as reader:
    for chunk in reader:
        df_nf = pd.concat([df_nf, chunk[chunk["Is Fraud?"] == "No"].sample(frac=0.05)])
        df_f = pd.concat([df_f, chunk[chunk["Is Fraud?"] == "Yes"]])
        vc = chunk["Is Fraud?"].value_counts()
        new = pd.DataFrame({"No": [vc[0]], "Yes": [vc[1]]})
        dist = pd.concat([dist, new])

df_nf.to_csv(f"{os.getenv('HOME')}/card_transactions_non-frauds.csv")
df_f.to_csv(f"{os.getenv('HOME')}/card_transactions_frauds.csv")
print(f"Ratio Fraud/Non-Fraud: {dist['Yes'].sum()/dist['No'].sum()}")
dist

Ratio Fraud/Non-Fraud: 0.00122169500749739


,No,Yes
0,999008.0,992.0
0,998768.0,1232.0
0,998803.0,1197.0
0,998856.0,1144.0
0,998891.0,1109.0
0,998675.0,1325.0
0,998830.0,1170.0
0,998686.0,1314.0
0,998749.0,1251.0
0,998721.0,1279.0


## 1a) Preprocess raw data in db2

In [5]:
from trino.dbapi import connect
import pandas as pd

conn = connect(host="trino.trino", port=8080, user="user")

cur = conn.cursor()
cur.execute("select * from jtopen.demo.fraud")
# df = pd.DataFrame(rows, columns=[i.name for i in cur.description])
# df.head()

In [6]:
import gc

chunk = 1_000_000
tmp_data_file = f"{os.getenv('HOME')}/card_transaction.tmp.csv"
with open(tmp_data_file, "w"):
    pass

test = 2
rows = []
cur_count = 0
first_iter = True
while True:
    cur_chunk = cur.fetchmany(chunk)
    if first_iter:
        columns = [i.name for i in cur.description]
        cur_chunk.insert(0, columns)
        first_iter = False
    if not cur_chunk:
        break
    with open(tmp_data_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(cur_chunk)
    gc.collect()
    cur_count += chunk
    print(f"cur row count: {cur_count}")

cur row count: 1000000
cur row count: 2000000
cur row count: 3000000
cur row count: 4000000
cur row count: 5000000
cur row count: 6000000
cur row count: 7000000
cur row count: 8000000
cur row count: 9000000
cur row count: 10000000
cur row count: 11000000
cur row count: 12000000
cur row count: 13000000
cur row count: 14000000
cur row count: 15000000
cur row count: 16000000
cur row count: 17000000
cur row count: 18000000
cur row count: 19000000
cur row count: 20000000
cur row count: 21000000
cur row count: 22000000
cur row count: 23000000
cur row count: 24000000
cur row count: 25000000


In [2]:
tmp_data_file = f"{os.getenv('HOME')}/card_transaction.tmp.csv"
# df_raw = pd.read_csv(tmp_data_file)
# len(df_raw)

In [27]:
df_raw.head()

,user,card,year,month,day,time,amount,use chip,merchant name,merchant city,merchant state,zip,mcc,errors?,is fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,...,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,...,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754,5651,...,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750,5912,...,No


In [3]:
import math
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    FunctionTransformer,
    MinMaxScaler,
    LabelBinarizer,
)
from sklearn_pandas import DataFrameMapper

v1_card_transactions = f"{os.getenv('HOME')}/card_transaction.v1.csv"


def timeEncoder(X):
    X_hm = X["time"].str.split(":", expand=True)
    d = pd.to_datetime(
        dict(
            year=X["year"], month=X["month"], day=X["day"], hour=X_hm[0], minute=X_hm[1]
        )
    ).astype(int)
    return pd.DataFrame(d)


def amtEncoder(X):
    amt = (
        X.apply(lambda x: x[1:])
        .astype(float)
        .map(lambda amt: max(1, amt))
        .map(math.log)
    )
    return pd.DataFrame(amt)


def decimalEncoder(X, length=5):
    dnew = pd.DataFrame()
    for i in range(length):
        dnew[i] = np.mod(X, 10)
        X = np.floor_divide(X, 10)
    return dnew


def fraudEncoder(X):
    return np.where(X == "Yes", 1, 0).astype(int)


# df_nf = pd.read_csv(f"{os.getenv('HOME')}/card_transactions_non-frauds.csv")
# df_f = pd.read_csv(f"{os.getenv('HOME')}/card_transactions_frauds.csv")
# tdf = pd.concat([df_nf, df_f])
tdf = pd.read_csv(tmp_data_file, nrows=1_000_000)
tdf.columns = map(str.lower, tdf.columns)
tdf["merchant name"] = tdf["merchant name"].astype(str)
tdf.drop(["mcc", "zip", "merchant state"], axis=1, inplace=True)
tdf.sort_values(by=["user", "card"], inplace=True)
tdf.reset_index(inplace=True, drop=True)

mapper = DataFrameMapper(
    [
        ("is fraud?", FunctionTransformer(fraudEncoder)),
        (
            "merchant name",
            [LabelEncoder(), FunctionTransformer(decimalEncoder), OneHotEncoder()],
        ),
        (
            "merchant city",
            [LabelEncoder(), FunctionTransformer(decimalEncoder), OneHotEncoder()],
        ),
        (["use chip"], [SimpleImputer(strategy="constant"), LabelBinarizer()]),
        (["errors?"], [SimpleImputer(strategy="constant"), LabelBinarizer()]),
        (
            ["year", "month", "day", "time"],
            [FunctionTransformer(timeEncoder), MinMaxScaler()],
        ),
        ("amount", [FunctionTransformer(amtEncoder), MinMaxScaler()]),
    ],
    input_df=True,
    df_out=True,
)
mapper.fit(tdf)
tdf = mapper.transform(tdf)

tdf.head()

,is fraud?,merchant name_0,merchant name_1,merchant name_2,merchant name_3,merchant name_4,merchant name_5,merchant name_6,merchant name_7,merchant name_8,...,"errors?_Bad Expiration,Technical Glitch,","errors?_Bad PIN,","errors?_Bad PIN,Insufficient Balance,","errors?_Bad PIN,Technical Glitch,","errors?_Bad Zipcode,","errors?_Insufficient Balance,","errors?_Insufficient Balance,Technical Glitch,","errors?_Technical Glitch,",year_month_day_time,amount
0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.268375,0.554906
1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.268377,0.413490
2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.268490,0.542650
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0.268544,0.550478
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.268604,0.526890


In [4]:
tdf["is fraud?"].value_counts()

0    999008
1       992
Name: is fraud?, dtype: int64

In [5]:
tdf.to_csv("./preprocessed_transactions.csv", index=False)

In [6]:
# build "CREATE TABLE" statement for postgresql
sql = "CREATE TABLE IF NOT EXISTS jtopen.demo.transactions ("


def get_dtype(d):
    if d == "float64":
        return "real"
    elif d == "int64":
        return "bigint"
    else:
        return "varchar"


for col, d in zip(tdf.columns, tdf.dtypes):
    sql += f'"{col.replace("?", "")}" {get_dtype(d)}, '
sql += ";"

In [7]:
sql = re.sub(r", ;$", ");", sql)
sql

'CREATE TABLE IF NOT EXISTS jtopen.demo.transactions ("is fraud" bigint, "merchant name_0" real, "merchant name_1" real, "merchant name_2" real, "merchant name_3" real, "merchant name_4" real, "merchant name_5" real, "merchant name_6" real, "merchant name_7" real, "merchant name_8" real, "merchant name_9" real, "merchant name_10" real, "merchant name_11" real, "merchant name_12" real, "merchant name_13" real, "merchant name_14" real, "merchant name_15" real, "merchant name_16" real, "merchant name_17" real, "merchant name_18" real, "merchant name_19" real, "merchant name_20" real, "merchant name_21" real, "merchant name_22" real, "merchant name_23" real, "merchant name_24" real, "merchant name_25" real, "merchant name_26" real, "merchant name_27" real, "merchant name_28" real, "merchant name_29" real, "merchant name_30" real, "merchant name_31" real, "merchant name_32" real, "merchant name_33" real, "merchant name_34" real, "merchant name_35" real, "merchant name_36" real, "merchant na

In [14]:
from trino.dbapi import connect
import pandas as pd


conn = connect(host="trino.trino", port=8080, user="user")

create_table = """
CREATE TABLE IF NOT EXISTS jtopen.demo.orders (
  orderkey INTEGER,
  orderstatus varchar(20)
)
"""

cur = conn.cursor()
cur.execute(create_table)

In [16]:
cur.execute("select * from jtopen.demo.orders")
rows = cur.fetchall()
rows
columns = [i.name for i in cur.description]
print(columns)

['orderkey', 'orderstatus']


In [18]:
cur.execute("show columns from jtopen.demo.fraud")
rows = cur.fetchall()
rows

[['user', 'decimal(4,0)', '', ''],
 ['card', 'decimal(1,0)', '', ''],
 ['year', 'decimal(4,0)', '', ''],
 ['month', 'decimal(2,0)', '', ''],
 ['day', 'decimal(2,0)', '', ''],
 ['time', 'char(5)', '', ''],
 ['amount', 'char(9)', '', ''],
 ['use chip', 'char(18)', '', ''],
 ['merchant name', 'decimal(37,0)', '', ''],
 ['merchant city', 'char(26)', '', ''],
 ['merchant state', 'char(32)', '', ''],
 ['zip', 'decimal(5,0)', '', ''],
 ['mcc', 'decimal(4,0)', '', ''],
 ['errors?', 'char(52)', '', ''],
 ['is fraud?', 'char(3)', '', '']]